In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from pytorch_tabnet.tab_model import TabNetRegressor

# --- Prepare Real Data ---
real_df = pd.read_csv("final_quality_dataset.csv")
target_col = "Rating (Out of 10)"
condition_col = "Company Location"  

# Features and target variable
X = real_df.drop(columns=[target_col])
y = real_df[target_col]

# Scaling the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# --- GAN Definitions ---

# Generator Model (generating both features and target)
class Generator(nn.Module):
    def __init__(self, input_dim, feature_dim, target_dim):
        super(Generator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, feature_dim + target_dim),  # Generate both features and target
            nn.Tanh()  # Output range from -1 to 1 (scaled features)
        )

    def forward(self, z):
        return self.fc(z)

# Discriminator Model (taking both features and target as input)
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()  # Output probability of real (1) or fake (0)
        )

    def forward(self, x):
        return self.fc(x)

# --- Training the GAN ---
def train_gan(real_data, target_data, num_epochs=100, batch_size=64, z_dim=100, lr=0.0002):
    # Initialize the models
    generator = Generator(input_dim=z_dim, feature_dim=real_data.shape[1], target_dim=1)
    discriminator = Discriminator(input_dim=real_data.shape[1] + 1)  # Features + target

    # Optimizers for both models
    optimizer_g = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    optimizer_d = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

    # Loss function
    criterion = nn.BCELoss()

    # DataLoader for real data
    real_data = torch.tensor(real_data, dtype=torch.float32)
    target_data = torch.tensor(target_data.values, dtype=torch.float32).view(-1, 1)  # Ensure target is 2D (n_samples, 1)
    real_labels = torch.ones(batch_size, 1)  # Real labels are 1

    # Train the GAN
    for epoch in range(num_epochs):
        for _ in range(real_data.shape[0] // batch_size):
            # Create a batch of real data
            indices = torch.randint(0, real_data.shape[0], (batch_size,))
            real_batch = real_data[indices]
            real_target_batch = target_data[indices]
            real_combined = torch.cat((real_batch, real_target_batch), dim=1)

            # Train the discriminator
            optimizer_d.zero_grad()
            z = torch.randn(batch_size, z_dim)
            fake_data = generator(z)  # Generate fake data
            fake_features = fake_data[:, :-1]
            fake_target = fake_data[:, -1].view(-1, 1)
            fake_combined = torch.cat((fake_features, fake_target), dim=1)
            fake_labels = torch.zeros(batch_size, 1)  # Fake labels are 0

            # Calculate discriminator loss for real and fake data
            real_loss = criterion(discriminator(real_combined), real_labels)
            fake_loss = criterion(discriminator(fake_combined.detach()), fake_labels)
            d_loss = real_loss + fake_loss
            d_loss.backward()
            optimizer_d.step()

            # Train the generator
            optimizer_g.zero_grad()
            g_loss = criterion(discriminator(fake_combined), real_labels)  # We want the generator to fool the discriminator
            g_loss.backward()
            optimizer_g.step()

        # Print the progress
        print(f'Epoch [{epoch+1}/{num_epochs}], D Loss: {d_loss.item()}, G Loss: {g_loss.item()}')

    # After training, return the generator
    return generator

# --- Generate Synthetic Data ---
def generate_synthetic_data(generator, num_samples=15000, z_dim=100):
    z = torch.randn(num_samples, z_dim)
    synthetic_data = generator(z).detach().numpy()
    return synthetic_data

# --- Train the GAN and Generate Data ---
gan_generator = train_gan(X_scaled, y, num_epochs=200)  # Train the GAN for 200 epochs
synthetic_data = generate_synthetic_data(gan_generator, num_samples=15000)

# --- Separate the Generated Features and Target ---
synthetic_df = pd.DataFrame(synthetic_data, columns=list(X.columns) + [target_col])

# --- Combine Real and Synthetic Data ---
combined_df = pd.concat([real_df, synthetic_df], ignore_index=True)
X_combined = combined_df.drop(columns=[target_col])
y_combined = combined_df[target_col]
X_combined_scaled = scaler.fit_transform(X_combined)

# --- Train Test Split ---
X_train, X_test, y_train, y_test = train_test_split(X_combined_scaled, y_combined, test_size=0.2, random_state=42)

# --- Initialize TabNet Model ---
tabnet = TabNetRegressor(
    n_d=64, n_a=64, n_steps=5, gamma=1.3,
    lambda_sparse=1e-4, optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=1e-3), mask_type='entmax',
    seed=42, verbose=0
)

# --- Define Evaluate Function ---
def evaluate_tabnet(X_train, y_train, X_test, y_test, source="Combined"):
    tabnet.fit(
        X_train=X_train, y_train=y_train.values.reshape(-1, 1),
        eval_set=[(X_test, y_test.values.reshape(-1, 1))],
        eval_name=["val"], eval_metric=["mae"],
        max_epochs=300, patience=30,
        batch_size=256, virtual_batch_size=64
    )

    # Predict and evaluate the model
    y_pred = tabnet.predict(X_test).flatten()
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mae_score_10pt = 10 * (1 - min(mae / 10, 1))
    print(f"\n TabNet Evaluation ({source}):")
    print(f"R² Score: {r2:.4f}")
    print(f"MAE: {mae:.2f}")
    print(f"MAE Score (out of 10): {mae_score_10pt:.2f}")

# --- Evaluate TabNet on Combined Dataset (Real + Synthetic) ---
evaluate_tabnet(X_train, y_train, X_test, y_test, source="Real + Synthetic")


Epoch [1/200], D Loss: 0.8577761650085449, G Loss: 0.6889827251434326
Epoch [2/200], D Loss: 0.725504994392395, G Loss: 0.7858401536941528
Epoch [3/200], D Loss: 0.5240496397018433, G Loss: 1.0229299068450928
Epoch [4/200], D Loss: 0.38092556595802307, G Loss: 1.4362369775772095
Epoch [5/200], D Loss: 0.23830361664295197, G Loss: 1.780081868171692
Epoch [6/200], D Loss: 0.15059837698936462, G Loss: 2.2080397605895996
Epoch [7/200], D Loss: 0.09455040842294693, G Loss: 2.709123373031616
Epoch [8/200], D Loss: 0.08571122586727142, G Loss: 2.988035202026367
Epoch [9/200], D Loss: 0.06464118510484695, G Loss: 3.015889883041382
Epoch [10/200], D Loss: 0.09518416225910187, G Loss: 3.4360032081604004
Epoch [11/200], D Loss: 0.034652095288038254, G Loss: 3.568364143371582
Epoch [12/200], D Loss: 0.04614299535751343, G Loss: 3.684086322784424
Epoch [13/200], D Loss: 0.025225352495908737, G Loss: 3.9328978061676025
Epoch [14/200], D Loss: 0.041199952363967896, G Loss: 4.096744537353516
Epoch [15

/opt/conda/miniconda3/envs/keras-torch-310/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



 TabNet Evaluation (Real + Synthetic):
R² Score: 0.9559
MAE: 0.10
MAE Score (out of 10): 9.90


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from pytorch_tabnet.tab_model import TabNetRegressor

# --- Prepare Real Data ---
real_df = pd.read_csv("final_quality_dataset.csv")
target_col = "Rating (Out of 10)"
condition_col = "Company Location"  

# Features and target variable
X = real_df.drop(columns=[target_col])
y = real_df[target_col]

# Scaling the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# --- GAN Definitions ---

# Generator Model (generating both features and target)
class Generator(nn.Module):
    def __init__(self, input_dim, feature_dim, target_dim):
        super(Generator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, feature_dim + target_dim),  # Generate both features and target
            nn.Tanh()  # Output range from -1 to 1 (scaled features)
        )

    def forward(self, z):
        return self.fc(z)

# Discriminator Model (taking both features and target as input)
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()  # Output probability of real (1) or fake (0)
        )

    def forward(self, x):
        return self.fc(x)

# --- Training the GAN ---
def train_gan(real_data, target_data, num_epochs=100, batch_size=64, z_dim=100, lr=0.0002):
    # Initialize the models
    generator = Generator(input_dim=z_dim, feature_dim=real_data.shape[1], target_dim=1)
    discriminator = Discriminator(input_dim=real_data.shape[1] + 1)  # Features + target

    # Optimizers for both models
    optimizer_g = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    optimizer_d = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

    # Loss function
    criterion = nn.BCELoss()

    # DataLoader for real data
    real_data = torch.tensor(real_data, dtype=torch.float32)
    target_data = torch.tensor(target_data.values, dtype=torch.float32).view(-1, 1)  # Ensure target is 2D (n_samples, 1)
    real_labels = torch.ones(batch_size, 1)  # Real labels are 1

    # Train the GAN
    for epoch in range(num_epochs):
        for _ in range(real_data.shape[0] // batch_size):
            # Create a batch of real data
            indices = torch.randint(0, real_data.shape[0], (batch_size,))
            real_batch = real_data[indices]
            real_target_batch = target_data[indices]
            real_combined = torch.cat((real_batch, real_target_batch), dim=1)

            # Train the discriminator
            optimizer_d.zero_grad()
            z = torch.randn(batch_size, z_dim)
            fake_data = generator(z)  # Generate fake data
            fake_features = fake_data[:, :-1]
            fake_target = fake_data[:, -1].view(-1, 1)
            fake_combined = torch.cat((fake_features, fake_target), dim=1)
            fake_labels = torch.zeros(batch_size, 1)  # Fake labels are 0

            # Calculate discriminator loss for real and fake data
            real_loss = criterion(discriminator(real_combined), real_labels)
            fake_loss = criterion(discriminator(fake_combined.detach()), fake_labels)
            d_loss = real_loss + fake_loss
            d_loss.backward()
            optimizer_d.step()

            # Train the generator
            optimizer_g.zero_grad()
            g_loss = criterion(discriminator(fake_combined), real_labels)  # We want the generator to fool the discriminator
            g_loss.backward()
            optimizer_g.step()

        # Print the progress
        print(f'Epoch [{epoch+1}/{num_epochs}], D Loss: {d_loss.item()}, G Loss: {g_loss.item()}')

    # After training, return the generator
    return generator

# --- Generate Synthetic Data ---
def generate_synthetic_data(generator, num_samples=15000, z_dim=100):
    z = torch.randn(num_samples, z_dim)
    synthetic_data = generator(z).detach().numpy()
    return synthetic_data

# --- Train the GAN and Generate Data ---
gan_generator = train_gan(X_scaled, y, num_epochs=200)  # Train the GAN for 200 epochs
synthetic_data = generate_synthetic_data(gan_generator, num_samples=15000)

# --- Separate the Generated Features and Target ---
synthetic_df = pd.DataFrame(synthetic_data, columns=list(X.columns) + [target_col])

# --- Combine the Real and Synthetic Data for Training ---
combined_df = pd.concat([real_df, synthetic_df], ignore_index=True)
X_combined = combined_df.drop(columns=[target_col])
y_combined = combined_df[target_col]
X_combined_scaled = scaler.fit_transform(X_combined)

# --- Train Test Split ---
X_train, X_test, y_train, y_test = train_test_split(X_combined_scaled, y_combined, test_size=0.2, random_state=42)

# --- Initialize TabNet Model ---
tabnet = TabNetRegressor(
    n_d=64, n_a=64, n_steps=5, gamma=1.3,
    lambda_sparse=1e-4, optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=1e-3), mask_type='entmax',
    seed=42, verbose=0
)

# --- Define Evaluate Function ---
def evaluate_tabnet(X_train, y_train, X_test, y_test, source="Real"):
    tabnet.fit(
        X_train=X_train, y_train=y_train.values.reshape(-1, 1),
        eval_set=[(X_test, y_test.values.reshape(-1, 1))],
        eval_name=["val"], eval_metric=["mae"],
        max_epochs=300, patience=30,
        batch_size=256, virtual_batch_size=64
    )

    # Predict and evaluate the model
    y_pred = tabnet.predict(X_test).flatten()
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mae_score_10pt = 10 * (1 - min(mae / 10, 1))
    print(f"\n TabNet Evaluation ({source}):")
    print(f"R² Score: {r2:.4f}")
    print(f"MAE: {mae:.2f}")
    print(f"MAE Score (out of 10): {mae_score_10pt:.2f}")

# --- Evaluate TabNet on Real Data Only (No Synthetic) ---
evaluate_tabnet(X_test, y_test, X_test, y_test, source="Real")


Epoch [1/200], D Loss: 0.907095730304718, G Loss: 0.6799144744873047
Epoch [2/200], D Loss: 0.7609828114509583, G Loss: 0.7733179926872253
Epoch [3/200], D Loss: 0.5169694423675537, G Loss: 1.0756999254226685
Epoch [4/200], D Loss: 0.32631993293762207, G Loss: 1.5450141429901123
Epoch [5/200], D Loss: 0.2105439305305481, G Loss: 1.8598569631576538
Epoch [6/200], D Loss: 0.1460232138633728, G Loss: 2.24847149848938
Epoch [7/200], D Loss: 0.12431879341602325, G Loss: 2.4079506397247314
Epoch [8/200], D Loss: 0.11471237242221832, G Loss: 2.900233268737793
Epoch [9/200], D Loss: 0.1088518500328064, G Loss: 2.9996566772460938
Epoch [10/200], D Loss: 0.04058520868420601, G Loss: 3.3436763286590576
Epoch [11/200], D Loss: 0.05659887194633484, G Loss: 3.4957544803619385
Epoch [12/200], D Loss: 0.04654412716627121, G Loss: 3.805135488510132
Epoch [13/200], D Loss: 0.03037361241877079, G Loss: 3.7924187183380127
Epoch [14/200], D Loss: 0.05941373482346535, G Loss: 3.8893187046051025
Epoch [15/20

/opt/conda/miniconda3/envs/keras-torch-310/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



 TabNet Evaluation (Real):
R² Score: 0.9512
MAE: 0.10
MAE Score (out of 10): 9.90


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from pytorch_tabnet.tab_model import TabNetRegressor

# --- Prepare Real Data ---
real_df = pd.read_csv("final_quality_dataset.csv")
target_col = "Rating (Out of 10)"
condition_col = "Bean Origin"  

# Features and target variable
X = real_df.drop(columns=[target_col])
y = real_df[target_col]

# Scaling the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# --- GAN Definitions ---

# Generator Model (generating both features and target)
class Generator(nn.Module):
    def __init__(self, input_dim, feature_dim, target_dim):
        super(Generator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, feature_dim + target_dim),  # Generate both features and target
            nn.Tanh()  # Output range from -1 to 1 (scaled features)
        )

    def forward(self, z):
        return self.fc(z)

# Discriminator Model (taking both features and target as input)
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()  # Output probability of real (1) or fake (0)
        )

    def forward(self, x):
        return self.fc(x)

# --- Training the GAN ---
def train_gan(real_data, target_data, num_epochs=100, batch_size=64, z_dim=100, lr=0.0002):
    # Initialize the models
    generator = Generator(input_dim=z_dim, feature_dim=real_data.shape[1], target_dim=1)
    discriminator = Discriminator(input_dim=real_data.shape[1] + 1)  # Features + target

    # Optimizers for both models
    optimizer_g = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    optimizer_d = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

    # Loss function
    criterion = nn.BCELoss()

    # DataLoader for real data
    real_data = torch.tensor(real_data, dtype=torch.float32)
    target_data = torch.tensor(target_data.values, dtype=torch.float32).view(-1, 1)  # Ensure target is 2D (n_samples, 1)
    real_labels = torch.ones(batch_size, 1)  # Real labels are 1

    # Train the GAN
    for epoch in range(num_epochs):
        for _ in range(real_data.shape[0] // batch_size):
            # Create a batch of real data
            indices = torch.randint(0, real_data.shape[0], (batch_size,))
            real_batch = real_data[indices]
            real_target_batch = target_data[indices]
            real_combined = torch.cat((real_batch, real_target_batch), dim=1)

            # Train the discriminator
            optimizer_d.zero_grad()
            z = torch.randn(batch_size, z_dim)
            fake_data = generator(z)  # Generate fake data
            fake_features = fake_data[:, :-1]
            fake_target = fake_data[:, -1].view(-1, 1)
            fake_combined = torch.cat((fake_features, fake_target), dim=1)
            fake_labels = torch.zeros(batch_size, 1)  # Fake labels are 0

            # Calculate discriminator loss for real and fake data
            real_loss = criterion(discriminator(real_combined), real_labels)
            fake_loss = criterion(discriminator(fake_combined.detach()), fake_labels)
            d_loss = real_loss + fake_loss
            d_loss.backward()
            optimizer_d.step()

            # Train the generator
            optimizer_g.zero_grad()
            g_loss = criterion(discriminator(fake_combined), real_labels)  # We want the generator to fool the discriminator
            g_loss.backward()
            optimizer_g.step()

        # Print the progress
        print(f'Epoch [{epoch+1}/{num_epochs}], D Loss: {d_loss.item()}, G Loss: {g_loss.item()}')

    # After training, return the generator
    return generator

# --- Generate Synthetic Data ---
def generate_synthetic_data(generator, num_samples=15000, z_dim=100):
    z = torch.randn(num_samples, z_dim)
    synthetic_data = generator(z).detach().numpy()
    return synthetic_data

# --- Train the GAN and Generate Data ---
gan_generator = train_gan(X_scaled, y, num_epochs=200)  # Train the GAN for 200 epochs
synthetic_data = generate_synthetic_data(gan_generator, num_samples=15000)

# --- Separate the Generated Features and Target ---
synthetic_df = pd.DataFrame(synthetic_data, columns=list(X.columns) + [target_col])

# --- Combine Real and Synthetic Data ---
combined_df = pd.concat([real_df, synthetic_df], ignore_index=True)
X_combined = combined_df.drop(columns=[target_col])
y_combined = combined_df[target_col]
X_combined_scaled = scaler.fit_transform(X_combined)

# --- Train Test Split ---
X_train, X_test, y_train, y_test = train_test_split(X_combined_scaled, y_combined, test_size=0.2, random_state=42)

# --- Initialize TabNet Model ---
tabnet = TabNetRegressor(
    n_d=64, n_a=64, n_steps=5, gamma=1.3,
    lambda_sparse=1e-4, optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=1e-3), mask_type='entmax',
    seed=42, verbose=0
)

# --- Define Evaluate Function ---
def evaluate_tabnet(X_train, y_train, X_test, y_test, source="Combined"):
    tabnet.fit(
        X_train=X_train, y_train=y_train.values.reshape(-1, 1),
        eval_set=[(X_test, y_test.values.reshape(-1, 1))],
        eval_name=["val"], eval_metric=["mae"],
        max_epochs=300, patience=30,
        batch_size=256, virtual_batch_size=64
    )

    # Predict and evaluate the model
    y_pred = tabnet.predict(X_test).flatten()
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mae_score_10pt = 10 * (1 - min(mae / 10, 1))
    print(f"\n TabNet Evaluation ({source}):")
    print(f"R² Score: {r2:.4f}")
    print(f"MAE: {mae:.2f}")
    print(f"MAE Score (out of 10): {mae_score_10pt:.2f}")

# --- Evaluate TabNet on Combined Dataset (Real + Synthetic) ---
evaluate_tabnet(X_train, y_train, X_test, y_test, source="Real + Synthetic")


Epoch [1/200], D Loss: 0.8778948783874512, G Loss: 0.6751813888549805
Epoch [2/200], D Loss: 0.7042200565338135, G Loss: 0.8367056250572205
Epoch [3/200], D Loss: 0.4560694992542267, G Loss: 1.1780732870101929
Epoch [4/200], D Loss: 0.3223233222961426, G Loss: 1.571795105934143
Epoch [5/200], D Loss: 0.1685745120048523, G Loss: 2.140671730041504
Epoch [6/200], D Loss: 0.14509174227714539, G Loss: 2.439729690551758
Epoch [7/200], D Loss: 0.13128970563411713, G Loss: 2.4896953105926514
Epoch [8/200], D Loss: 0.07440848648548126, G Loss: 3.3049235343933105
Epoch [9/200], D Loss: 0.10969997942447662, G Loss: 3.2235665321350098
Epoch [10/200], D Loss: 0.04621899873018265, G Loss: 3.368377685546875
Epoch [11/200], D Loss: 0.12146667391061783, G Loss: 3.6299355030059814
Epoch [12/200], D Loss: 0.022981680929660797, G Loss: 3.900304079055786
Epoch [13/200], D Loss: 0.021858390420675278, G Loss: 4.039379119873047
Epoch [14/200], D Loss: 0.03873839229345322, G Loss: 4.105579376220703
Epoch [15/2

/opt/conda/miniconda3/envs/keras-torch-310/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



 TabNet Evaluation (Real + Synthetic):
R² Score: 0.9502
MAE: 0.10
MAE Score (out of 10): 9.90


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from pytorch_tabnet.tab_model import TabNetRegressor

# --- Prepare Real Data ---
real_df = pd.read_csv("final_quality_dataset.csv")
target_col = "Rating (Out of 10)"
condition_col = "Bean Origin"  

# Features and target variable
X = real_df.drop(columns=[target_col])
y = real_df[target_col]

# Scaling the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# --- GAN Definitions ---

# Generator Model (generating both features and target)
class Generator(nn.Module):
    def __init__(self, input_dim, feature_dim, target_dim):
        super(Generator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, feature_dim + target_dim),  # Generate both features and target
            nn.Tanh()  # Output range from -1 to 1 (scaled features)
        )

    def forward(self, z):
        return self.fc(z)

# Discriminator Model (taking both features and target as input)
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()  # Output probability of real (1) or fake (0)
        )

    def forward(self, x):
        return self.fc(x)

# --- Training the GAN ---
def train_gan(real_data, target_data, num_epochs=100, batch_size=64, z_dim=100, lr=0.0002):
    # Initialize the models
    generator = Generator(input_dim=z_dim, feature_dim=real_data.shape[1], target_dim=1)
    discriminator = Discriminator(input_dim=real_data.shape[1] + 1)  # Features + target

    # Optimizers for both models
    optimizer_g = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    optimizer_d = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

    # Loss function
    criterion = nn.BCELoss()

    # DataLoader for real data
    real_data = torch.tensor(real_data, dtype=torch.float32)
    target_data = torch.tensor(target_data.values, dtype=torch.float32).view(-1, 1)  # Ensure target is 2D (n_samples, 1)
    real_labels = torch.ones(batch_size, 1)  # Real labels are 1

    # Train the GAN
    for epoch in range(num_epochs):
        for _ in range(real_data.shape[0] // batch_size):
            # Create a batch of real data
            indices = torch.randint(0, real_data.shape[0], (batch_size,))
            real_batch = real_data[indices]
            real_target_batch = target_data[indices]
            real_combined = torch.cat((real_batch, real_target_batch), dim=1)

            # Train the discriminator
            optimizer_d.zero_grad()
            z = torch.randn(batch_size, z_dim)
            fake_data = generator(z)  # Generate fake data
            fake_features = fake_data[:, :-1]
            fake_target = fake_data[:, -1].view(-1, 1)
            fake_combined = torch.cat((fake_features, fake_target), dim=1)
            fake_labels = torch.zeros(batch_size, 1)  # Fake labels are 0

            # Calculate discriminator loss for real and fake data
            real_loss = criterion(discriminator(real_combined), real_labels)
            fake_loss = criterion(discriminator(fake_combined.detach()), fake_labels)
            d_loss = real_loss + fake_loss
            d_loss.backward()
            optimizer_d.step()

            # Train the generator
            optimizer_g.zero_grad()
            g_loss = criterion(discriminator(fake_combined), real_labels)  # We want the generator to fool the discriminator
            g_loss.backward()
            optimizer_g.step()

        # Print the progress
        print(f'Epoch [{epoch+1}/{num_epochs}], D Loss: {d_loss.item()}, G Loss: {g_loss.item()}')

    # After training, return the generator
    return generator

# --- Generate Synthetic Data ---
def generate_synthetic_data(generator, num_samples=15000, z_dim=100):
    z = torch.randn(num_samples, z_dim)
    synthetic_data = generator(z).detach().numpy()
    return synthetic_data

# --- Train the GAN and Generate Data ---
gan_generator = train_gan(X_scaled, y, num_epochs=200)  # Train the GAN for 200 epochs
synthetic_data = generate_synthetic_data(gan_generator, num_samples=15000)

# --- Separate the Generated Features and Target ---
synthetic_df = pd.DataFrame(synthetic_data, columns=list(X.columns) + [target_col])

# --- Combine the Real and Synthetic Data for Training ---
combined_df = pd.concat([real_df, synthetic_df], ignore_index=True)
X_combined = combined_df.drop(columns=[target_col])
y_combined = combined_df[target_col]
X_combined_scaled = scaler.fit_transform(X_combined)

# --- Train Test Split ---
X_train, X_test, y_train, y_test = train_test_split(X_combined_scaled, y_combined, test_size=0.2, random_state=42)

# --- Initialize TabNet Model ---
tabnet = TabNetRegressor(
    n_d=64, n_a=64, n_steps=5, gamma=1.3,
    lambda_sparse=1e-4, optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=1e-3), mask_type='entmax',
    seed=42, verbose=0
)

# --- Define Evaluate Function ---
def evaluate_tabnet(X_train, y_train, X_test, y_test, source="Real"):
    tabnet.fit(
        X_train=X_train, y_train=y_train.values.reshape(-1, 1),
        eval_set=[(X_test, y_test.values.reshape(-1, 1))],
        eval_name=["val"], eval_metric=["mae"],
        max_epochs=300, patience=30,
        batch_size=256, virtual_batch_size=64
    )

    # Predict and evaluate the model
    y_pred = tabnet.predict(X_test).flatten()
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mae_score_10pt = 10 * (1 - min(mae / 10, 1))
    print(f"\n TabNet Evaluation ({source}):")
    print(f"R² Score: {r2:.4f}")
    print(f"MAE: {mae:.2f}")
    print(f"MAE Score (out of 10): {mae_score_10pt:.2f}")

# --- Evaluate TabNet on Real Data Only (No Synthetic) ---
evaluate_tabnet(X_test, y_test, X_test, y_test, source="Real")


Epoch [1/200], D Loss: 0.8743411898612976, G Loss: 0.6827710270881653
Epoch [2/200], D Loss: 0.7126411199569702, G Loss: 0.8149493336677551
Epoch [3/200], D Loss: 0.49518948793411255, G Loss: 1.1082574129104614
Epoch [4/200], D Loss: 0.31662052869796753, G Loss: 1.4928334951400757
Epoch [5/200], D Loss: 0.20789305865764618, G Loss: 1.9123713970184326
Epoch [6/200], D Loss: 0.16528606414794922, G Loss: 2.2440414428710938
Epoch [7/200], D Loss: 0.09425505250692368, G Loss: 2.6288082599639893
Epoch [8/200], D Loss: 0.07108113169670105, G Loss: 2.823291063308716
Epoch [9/200], D Loss: 0.04915745556354523, G Loss: 3.1575539112091064
Epoch [10/200], D Loss: 0.10000951588153839, G Loss: 3.2547435760498047
Epoch [11/200], D Loss: 0.05519648641347885, G Loss: 3.672116279602051
Epoch [12/200], D Loss: 0.08757425099611282, G Loss: 3.7026290893554688
Epoch [13/200], D Loss: 0.02366252802312374, G Loss: 3.9585421085357666
Epoch [14/200], D Loss: 0.04194729030132294, G Loss: 3.9664220809936523
Epoch

/opt/conda/miniconda3/envs/keras-torch-310/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



 TabNet Evaluation (Real):
R² Score: 0.9579
MAE: 0.09
MAE Score (out of 10): 9.91


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from pytorch_tabnet.tab_model import TabNetRegressor

# --- Prepare Real Data ---
real_df = pd.read_csv("final_quality_dataset.csv")
target_col = "Rating (Out of 10)"
condition_col = "Cocoa Percent"  

# Features and target variable
X = real_df.drop(columns=[target_col])
y = real_df[target_col]

# Scaling the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# --- GAN Definitions ---

# Generator Model (generating both features and target)
class Generator(nn.Module):
    def __init__(self, input_dim, feature_dim, target_dim):
        super(Generator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, feature_dim + target_dim),  # Generate both features and target
            nn.Tanh()  # Output range from -1 to 1 (scaled features)
        )

    def forward(self, z):
        return self.fc(z)

# Discriminator Model (taking both features and target as input)
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()  # Output probability of real (1) or fake (0)
        )

    def forward(self, x):
        return self.fc(x)

# --- Training the GAN ---
def train_gan(real_data, target_data, num_epochs=100, batch_size=64, z_dim=100, lr=0.0002):
    # Initialize the models
    generator = Generator(input_dim=z_dim, feature_dim=real_data.shape[1], target_dim=1)
    discriminator = Discriminator(input_dim=real_data.shape[1] + 1)  # Features + target

    # Optimizers for both models
    optimizer_g = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    optimizer_d = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

    # Loss function
    criterion = nn.BCELoss()

    # DataLoader for real data
    real_data = torch.tensor(real_data, dtype=torch.float32)
    target_data = torch.tensor(target_data.values, dtype=torch.float32).view(-1, 1)  # Ensure target is 2D (n_samples, 1)
    real_labels = torch.ones(batch_size, 1)  # Real labels are 1

    # Train the GAN
    for epoch in range(num_epochs):
        for _ in range(real_data.shape[0] // batch_size):
            # Create a batch of real data
            indices = torch.randint(0, real_data.shape[0], (batch_size,))
            real_batch = real_data[indices]
            real_target_batch = target_data[indices]
            real_combined = torch.cat((real_batch, real_target_batch), dim=1)

            # Train the discriminator
            optimizer_d.zero_grad()
            z = torch.randn(batch_size, z_dim)
            fake_data = generator(z)  # Generate fake data
            fake_features = fake_data[:, :-1]
            fake_target = fake_data[:, -1].view(-1, 1)
            fake_combined = torch.cat((fake_features, fake_target), dim=1)
            fake_labels = torch.zeros(batch_size, 1)  # Fake labels are 0

            # Calculate discriminator loss for real and fake data
            real_loss = criterion(discriminator(real_combined), real_labels)
            fake_loss = criterion(discriminator(fake_combined.detach()), fake_labels)
            d_loss = real_loss + fake_loss
            d_loss.backward()
            optimizer_d.step()

            # Train the generator
            optimizer_g.zero_grad()
            g_loss = criterion(discriminator(fake_combined), real_labels)  # We want the generator to fool the discriminator
            g_loss.backward()
            optimizer_g.step()

        # Print the progress
        print(f'Epoch [{epoch+1}/{num_epochs}], D Loss: {d_loss.item()}, G Loss: {g_loss.item()}')

    # After training, return the generator
    return generator

# --- Generate Synthetic Data ---
def generate_synthetic_data(generator, num_samples=15000, z_dim=100):
    z = torch.randn(num_samples, z_dim)
    synthetic_data = generator(z).detach().numpy()
    return synthetic_data

# --- Train the GAN and Generate Data ---
gan_generator = train_gan(X_scaled, y, num_epochs=200)  # Train the GAN for 200 epochs
synthetic_data = generate_synthetic_data(gan_generator, num_samples=15000)

# --- Separate the Generated Features and Target ---
synthetic_df = pd.DataFrame(synthetic_data, columns=list(X.columns) + [target_col])

# --- Combine Real and Synthetic Data ---
combined_df = pd.concat([real_df, synthetic_df], ignore_index=True)
X_combined = combined_df.drop(columns=[target_col])
y_combined = combined_df[target_col]
X_combined_scaled = scaler.fit_transform(X_combined)

# --- Train Test Split ---
X_train, X_test, y_train, y_test = train_test_split(X_combined_scaled, y_combined, test_size=0.2, random_state=42)

# --- Initialize TabNet Model ---
tabnet = TabNetRegressor(
    n_d=64, n_a=64, n_steps=5, gamma=1.3,
    lambda_sparse=1e-4, optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=1e-3), mask_type='entmax',
    seed=42, verbose=0
)

# --- Define Evaluate Function ---
def evaluate_tabnet(X_train, y_train, X_test, y_test, source="Combined"):
    tabnet.fit(
        X_train=X_train, y_train=y_train.values.reshape(-1, 1),
        eval_set=[(X_test, y_test.values.reshape(-1, 1))],
        eval_name=["val"], eval_metric=["mae"],
        max_epochs=300, patience=30,
        batch_size=256, virtual_batch_size=64
    )

    # Predict and evaluate the model
    y_pred = tabnet.predict(X_test).flatten()
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mae_score_10pt = 10 * (1 - min(mae / 10, 1))
    print(f"\n TabNet Evaluation ({source}):")
    print(f"R² Score: {r2:.4f}")
    print(f"MAE: {mae:.2f}")
    print(f"MAE Score (out of 10): {mae_score_10pt:.2f}")

# --- Evaluate TabNet on Combined Dataset (Real + Synthetic) ---
evaluate_tabnet(X_train, y_train, X_test, y_test, source="Real + Synthetic")


Epoch [1/200], D Loss: 0.8125755786895752, G Loss: 0.7324211597442627
Epoch [2/200], D Loss: 0.6642315983772278, G Loss: 0.8722753524780273
Epoch [3/200], D Loss: 0.5491277575492859, G Loss: 1.0232934951782227
Epoch [4/200], D Loss: 0.3024533987045288, G Loss: 1.5476922988891602
Epoch [5/200], D Loss: 0.18403704464435577, G Loss: 2.067284107208252
Epoch [6/200], D Loss: 0.12922102212905884, G Loss: 2.3323678970336914
Epoch [7/200], D Loss: 0.08825168013572693, G Loss: 2.6518454551696777
Epoch [8/200], D Loss: 0.07346317917108536, G Loss: 2.8596627712249756
Epoch [9/200], D Loss: 0.0494958758354187, G Loss: 3.167191982269287
Epoch [10/200], D Loss: 0.06041761115193367, G Loss: 3.418583393096924
Epoch [11/200], D Loss: 0.04102315381169319, G Loss: 3.5002307891845703
Epoch [12/200], D Loss: 0.08679330348968506, G Loss: 3.8980565071105957
Epoch [13/200], D Loss: 0.04589375853538513, G Loss: 3.8395578861236572
Epoch [14/200], D Loss: 0.020029308274388313, G Loss: 3.9971961975097656
Epoch [1

/opt/conda/miniconda3/envs/keras-torch-310/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



 TabNet Evaluation (Real + Synthetic):
R² Score: 0.9537
MAE: 0.10
MAE Score (out of 10): 9.90


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from pytorch_tabnet.tab_model import TabNetRegressor

# --- Prepare Real Data ---
real_df = pd.read_csv("final_quality_dataset.csv")
target_col = "Rating (Out of 10)"
condition_col = "Cocoa Percent"  

# Features and target variable
X = real_df.drop(columns=[target_col])
y = real_df[target_col]

# Scaling the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# --- GAN Definitions ---

# Generator Model (generating both features and target)
class Generator(nn.Module):
    def __init__(self, input_dim, feature_dim, target_dim):
        super(Generator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, feature_dim + target_dim),  # Generate both features and target
            nn.Tanh()  # Output range from -1 to 1 (scaled features)
        )

    def forward(self, z):
        return self.fc(z)

# Discriminator Model (taking both features and target as input)
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()  # Output probability of real (1) or fake (0)
        )

    def forward(self, x):
        return self.fc(x)

# --- Training the GAN ---
def train_gan(real_data, target_data, num_epochs=100, batch_size=64, z_dim=100, lr=0.0002):
    # Initialize the models
    generator = Generator(input_dim=z_dim, feature_dim=real_data.shape[1], target_dim=1)
    discriminator = Discriminator(input_dim=real_data.shape[1] + 1)  # Features + target

    # Optimizers for both models
    optimizer_g = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    optimizer_d = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

    # Loss function
    criterion = nn.BCELoss()

    # DataLoader for real data
    real_data = torch.tensor(real_data, dtype=torch.float32)
    target_data = torch.tensor(target_data.values, dtype=torch.float32).view(-1, 1)  # Ensure target is 2D (n_samples, 1)
    real_labels = torch.ones(batch_size, 1)  # Real labels are 1

    # Train the GAN
    for epoch in range(num_epochs):
        for _ in range(real_data.shape[0] // batch_size):
            # Create a batch of real data
            indices = torch.randint(0, real_data.shape[0], (batch_size,))
            real_batch = real_data[indices]
            real_target_batch = target_data[indices]
            real_combined = torch.cat((real_batch, real_target_batch), dim=1)

            # Train the discriminator
            optimizer_d.zero_grad()
            z = torch.randn(batch_size, z_dim)
            fake_data = generator(z)  # Generate fake data
            fake_features = fake_data[:, :-1]
            fake_target = fake_data[:, -1].view(-1, 1)
            fake_combined = torch.cat((fake_features, fake_target), dim=1)
            fake_labels = torch.zeros(batch_size, 1)  # Fake labels are 0

            # Calculate discriminator loss for real and fake data
            real_loss = criterion(discriminator(real_combined), real_labels)
            fake_loss = criterion(discriminator(fake_combined.detach()), fake_labels)
            d_loss = real_loss + fake_loss
            d_loss.backward()
            optimizer_d.step()

            # Train the generator
            optimizer_g.zero_grad()
            g_loss = criterion(discriminator(fake_combined), real_labels)  # We want the generator to fool the discriminator
            g_loss.backward()
            optimizer_g.step()

        # Print the progress
        print(f'Epoch [{epoch+1}/{num_epochs}], D Loss: {d_loss.item()}, G Loss: {g_loss.item()}')

    # After training, return the generator
    return generator

# --- Generate Synthetic Data ---
def generate_synthetic_data(generator, num_samples=15000, z_dim=100):
    z = torch.randn(num_samples, z_dim)
    synthetic_data = generator(z).detach().numpy()
    return synthetic_data

# --- Train the GAN and Generate Data ---
gan_generator = train_gan(X_scaled, y, num_epochs=200)  # Train the GAN for 200 epochs
synthetic_data = generate_synthetic_data(gan_generator, num_samples=15000)

# --- Separate the Generated Features and Target ---
synthetic_df = pd.DataFrame(synthetic_data, columns=list(X.columns) + [target_col])

# --- Combine the Real and Synthetic Data for Training ---
combined_df = pd.concat([real_df, synthetic_df], ignore_index=True)
X_combined = combined_df.drop(columns=[target_col])
y_combined = combined_df[target_col]
X_combined_scaled = scaler.fit_transform(X_combined)

# --- Train Test Split ---
X_train, X_test, y_train, y_test = train_test_split(X_combined_scaled, y_combined, test_size=0.2, random_state=42)

# --- Initialize TabNet Model ---
tabnet = TabNetRegressor(
    n_d=64, n_a=64, n_steps=5, gamma=1.3,
    lambda_sparse=1e-4, optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=1e-3), mask_type='entmax',
    seed=42, verbose=0
)

# --- Define Evaluate Function ---
def evaluate_tabnet(X_train, y_train, X_test, y_test, source="Real"):
    tabnet.fit(
        X_train=X_train, y_train=y_train.values.reshape(-1, 1),
        eval_set=[(X_test, y_test.values.reshape(-1, 1))],
        eval_name=["val"], eval_metric=["mae"],
        max_epochs=300, patience=30,
        batch_size=256, virtual_batch_size=64
    )

    # Predict and evaluate the model
    y_pred = tabnet.predict(X_test).flatten()
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mae_score_10pt = 10 * (1 - min(mae / 10, 1))
    print(f"\n TabNet Evaluation ({source}):")
    print(f"R² Score: {r2:.4f}")
    print(f"MAE: {mae:.2f}")
    print(f"MAE Score (out of 10): {mae_score_10pt:.2f}")

# --- Evaluate TabNet on Real Data Only (No Synthetic) ---
evaluate_tabnet(X_test, y_test, X_test, y_test, source="Real")


Epoch [1/200], D Loss: 0.8488890528678894, G Loss: 0.7553848028182983
Epoch [2/200], D Loss: 0.6463993191719055, G Loss: 0.8874586224555969
Epoch [3/200], D Loss: 0.45654112100601196, G Loss: 1.2408074140548706
Epoch [4/200], D Loss: 0.28949594497680664, G Loss: 1.602854609489441
Epoch [5/200], D Loss: 0.17463213205337524, G Loss: 2.102020740509033
Epoch [6/200], D Loss: 0.11451518535614014, G Loss: 2.384799003601074
Epoch [7/200], D Loss: 0.08901240676641464, G Loss: 2.769280433654785
Epoch [8/200], D Loss: 0.08862604200839996, G Loss: 2.9502358436584473
Epoch [9/200], D Loss: 0.046764619648456573, G Loss: 3.207645893096924
Epoch [10/200], D Loss: 0.045644327998161316, G Loss: 3.2720537185668945
Epoch [11/200], D Loss: 0.027030162513256073, G Loss: 3.7895631790161133
Epoch [12/200], D Loss: 0.024912521243095398, G Loss: 3.964243173599243
Epoch [13/200], D Loss: 0.02685355767607689, G Loss: 3.753427028656006
Epoch [14/200], D Loss: 0.04543333128094673, G Loss: 3.945667028427124
Epoch [

/opt/conda/miniconda3/envs/keras-torch-310/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



 TabNet Evaluation (Real):
R² Score: 0.9548
MAE: 0.11
MAE Score (out of 10): 9.89


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from pytorch_tabnet.tab_model import TabNetRegressor

# --- Prepare Real Data ---
real_df = pd.read_csv("final_quality_dataset.csv")
target_col = "Rating (Out of 10)"
condition_col = "Temperature change on land Value in °c"  

# Features and target variable
X = real_df.drop(columns=[target_col])
y = real_df[target_col]

# Scaling the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# --- GAN Definitions ---

# Generator Model (generating both features and target)
class Generator(nn.Module):
    def __init__(self, input_dim, feature_dim, target_dim):
        super(Generator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, feature_dim + target_dim),  # Generate both features and target
            nn.Tanh()  # Output range from -1 to 1 (scaled features)
        )

    def forward(self, z):
        return self.fc(z)

# Discriminator Model (taking both features and target as input)
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()  # Output probability of real (1) or fake (0)
        )

    def forward(self, x):
        return self.fc(x)

# --- Training the GAN ---
def train_gan(real_data, target_data, num_epochs=100, batch_size=64, z_dim=100, lr=0.0002):
    # Initialize the models
    generator = Generator(input_dim=z_dim, feature_dim=real_data.shape[1], target_dim=1)
    discriminator = Discriminator(input_dim=real_data.shape[1] + 1)  # Features + target

    # Optimizers for both models
    optimizer_g = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    optimizer_d = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

    # Loss function
    criterion = nn.BCELoss()

    # DataLoader for real data
    real_data = torch.tensor(real_data, dtype=torch.float32)
    target_data = torch.tensor(target_data.values, dtype=torch.float32).view(-1, 1)  # Ensure target is 2D (n_samples, 1)
    real_labels = torch.ones(batch_size, 1)  # Real labels are 1

    # Train the GAN
    for epoch in range(num_epochs):
        for _ in range(real_data.shape[0] // batch_size):
            # Create a batch of real data
            indices = torch.randint(0, real_data.shape[0], (batch_size,))
            real_batch = real_data[indices]
            real_target_batch = target_data[indices]
            real_combined = torch.cat((real_batch, real_target_batch), dim=1)

            # Train the discriminator
            optimizer_d.zero_grad()
            z = torch.randn(batch_size, z_dim)
            fake_data = generator(z)  # Generate fake data
            fake_features = fake_data[:, :-1]
            fake_target = fake_data[:, -1].view(-1, 1)
            fake_combined = torch.cat((fake_features, fake_target), dim=1)
            fake_labels = torch.zeros(batch_size, 1)  # Fake labels are 0

            # Calculate discriminator loss for real and fake data
            real_loss = criterion(discriminator(real_combined), real_labels)
            fake_loss = criterion(discriminator(fake_combined.detach()), fake_labels)
            d_loss = real_loss + fake_loss
            d_loss.backward()
            optimizer_d.step()

            # Train the generator
            optimizer_g.zero_grad()
            g_loss = criterion(discriminator(fake_combined), real_labels)  # We want the generator to fool the discriminator
            g_loss.backward()
            optimizer_g.step()

        # Print the progress
        print(f'Epoch [{epoch+1}/{num_epochs}], D Loss: {d_loss.item()}, G Loss: {g_loss.item()}')

    # After training, return the generator
    return generator

# --- Generate Synthetic Data ---
def generate_synthetic_data(generator, num_samples=15000, z_dim=100):
    z = torch.randn(num_samples, z_dim)
    synthetic_data = generator(z).detach().numpy()
    return synthetic_data

# --- Train the GAN and Generate Data ---
gan_generator = train_gan(X_scaled, y, num_epochs=200)  # Train the GAN for 200 epochs
synthetic_data = generate_synthetic_data(gan_generator, num_samples=15000)

# --- Separate the Generated Features and Target ---
synthetic_df = pd.DataFrame(synthetic_data, columns=list(X.columns) + [target_col])

# --- Combine Real and Synthetic Data ---
combined_df = pd.concat([real_df, synthetic_df], ignore_index=True)
X_combined = combined_df.drop(columns=[target_col])
y_combined = combined_df[target_col]
X_combined_scaled = scaler.fit_transform(X_combined)

# --- Train Test Split ---
X_train, X_test, y_train, y_test = train_test_split(X_combined_scaled, y_combined, test_size=0.2, random_state=42)

# --- Initialize TabNet Model ---
tabnet = TabNetRegressor(
    n_d=64, n_a=64, n_steps=5, gamma=1.3,
    lambda_sparse=1e-4, optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=1e-3), mask_type='entmax',
    seed=42, verbose=0
)

# --- Define Evaluate Function ---
def evaluate_tabnet(X_train, y_train, X_test, y_test, source="Combined"):
    tabnet.fit(
        X_train=X_train, y_train=y_train.values.reshape(-1, 1),
        eval_set=[(X_test, y_test.values.reshape(-1, 1))],
        eval_name=["val"], eval_metric=["mae"],
        max_epochs=300, patience=30,
        batch_size=256, virtual_batch_size=64
    )

    # Predict and evaluate the model
    y_pred = tabnet.predict(X_test).flatten()
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mae_score_10pt = 10 * (1 - min(mae / 10, 1))
    print(f"\n TabNet Evaluation ({source}):")
    print(f"R² Score: {r2:.4f}")
    print(f"MAE: {mae:.2f}")
    print(f"MAE Score (out of 10): {mae_score_10pt:.2f}")

# --- Evaluate TabNet on Combined Dataset (Real + Synthetic) ---
evaluate_tabnet(X_train, y_train, X_test, y_test, source="Real + Synthetic")


Epoch [1/200], D Loss: 0.866467297077179, G Loss: 0.7148685455322266
Epoch [2/200], D Loss: 0.6984000205993652, G Loss: 0.8490974307060242
Epoch [3/200], D Loss: 0.49252232909202576, G Loss: 1.1085081100463867
Epoch [4/200], D Loss: 0.2727496922016144, G Loss: 1.6757904291152954
Epoch [5/200], D Loss: 0.17862577736377716, G Loss: 2.0776968002319336
Epoch [6/200], D Loss: 0.15282569825649261, G Loss: 2.402438163757324
Epoch [7/200], D Loss: 0.12366021424531937, G Loss: 2.925959587097168
Epoch [8/200], D Loss: 0.10227510333061218, G Loss: 3.1215786933898926
Epoch [9/200], D Loss: 0.05031714215874672, G Loss: 3.2981934547424316
Epoch [10/200], D Loss: 0.04280310124158859, G Loss: 3.5094563961029053
Epoch [11/200], D Loss: 0.14058619737625122, G Loss: 3.6631407737731934
Epoch [12/200], D Loss: 0.08426593989133835, G Loss: 3.9140477180480957
Epoch [13/200], D Loss: 0.08164574950933456, G Loss: 3.988262891769409
Epoch [14/200], D Loss: 0.02368675172328949, G Loss: 4.0489373207092285
Epoch [1

/opt/conda/miniconda3/envs/keras-torch-310/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



 TabNet Evaluation (Real + Synthetic):
R² Score: 0.9512
MAE: 0.10
MAE Score (out of 10): 9.90


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from pytorch_tabnet.tab_model import TabNetRegressor

# --- Prepare Real Data ---
real_df = pd.read_csv("final_quality_dataset.csv")
target_col = "Rating (Out of 10)"
condition_col = "Temperature change on land Value in °c"  

# Features and target variable
X = real_df.drop(columns=[target_col])
y = real_df[target_col]

# Scaling the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# --- GAN Definitions ---

# Generator Model (generating both features and target)
class Generator(nn.Module):
    def __init__(self, input_dim, feature_dim, target_dim):
        super(Generator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, feature_dim + target_dim),  # Generate both features and target
            nn.Tanh()  # Output range from -1 to 1 (scaled features)
        )

    def forward(self, z):
        return self.fc(z)

# Discriminator Model (taking both features and target as input)
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()  # Output probability of real (1) or fake (0)
        )

    def forward(self, x):
        return self.fc(x)

# --- Training the GAN ---
def train_gan(real_data, target_data, num_epochs=100, batch_size=64, z_dim=100, lr=0.0002):
    # Initialize the models
    generator = Generator(input_dim=z_dim, feature_dim=real_data.shape[1], target_dim=1)
    discriminator = Discriminator(input_dim=real_data.shape[1] + 1)  # Features + target

    # Optimizers for both models
    optimizer_g = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    optimizer_d = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

    # Loss function
    criterion = nn.BCELoss()

    # DataLoader for real data
    real_data = torch.tensor(real_data, dtype=torch.float32)
    target_data = torch.tensor(target_data.values, dtype=torch.float32).view(-1, 1)  # Ensure target is 2D (n_samples, 1)
    real_labels = torch.ones(batch_size, 1)  # Real labels are 1

    # Train the GAN
    for epoch in range(num_epochs):
        for _ in range(real_data.shape[0] // batch_size):
            # Create a batch of real data
            indices = torch.randint(0, real_data.shape[0], (batch_size,))
            real_batch = real_data[indices]
            real_target_batch = target_data[indices]
            real_combined = torch.cat((real_batch, real_target_batch), dim=1)

            # Train the discriminator
            optimizer_d.zero_grad()
            z = torch.randn(batch_size, z_dim)
            fake_data = generator(z)  # Generate fake data
            fake_features = fake_data[:, :-1]
            fake_target = fake_data[:, -1].view(-1, 1)
            fake_combined = torch.cat((fake_features, fake_target), dim=1)
            fake_labels = torch.zeros(batch_size, 1)  # Fake labels are 0

            # Calculate discriminator loss for real and fake data
            real_loss = criterion(discriminator(real_combined), real_labels)
            fake_loss = criterion(discriminator(fake_combined.detach()), fake_labels)
            d_loss = real_loss + fake_loss
            d_loss.backward()
            optimizer_d.step()

            # Train the generator
            optimizer_g.zero_grad()
            g_loss = criterion(discriminator(fake_combined), real_labels)  # We want the generator to fool the discriminator
            g_loss.backward()
            optimizer_g.step()

        # Print the progress
        print(f'Epoch [{epoch+1}/{num_epochs}], D Loss: {d_loss.item()}, G Loss: {g_loss.item()}')

    # After training, return the generator
    return generator

# --- Generate Synthetic Data ---
def generate_synthetic_data(generator, num_samples=15000, z_dim=100):
    z = torch.randn(num_samples, z_dim)
    synthetic_data = generator(z).detach().numpy()
    return synthetic_data

# --- Train the GAN and Generate Data ---
gan_generator = train_gan(X_scaled, y, num_epochs=200)  # Train the GAN for 200 epochs
synthetic_data = generate_synthetic_data(gan_generator, num_samples=15000)

# --- Separate the Generated Features and Target ---
synthetic_df = pd.DataFrame(synthetic_data, columns=list(X.columns) + [target_col])

# --- Combine the Real and Synthetic Data for Training ---
combined_df = pd.concat([real_df, synthetic_df], ignore_index=True)
X_combined = combined_df.drop(columns=[target_col])
y_combined = combined_df[target_col]
X_combined_scaled = scaler.fit_transform(X_combined)

# --- Train Test Split ---
X_train, X_test, y_train, y_test = train_test_split(X_combined_scaled, y_combined, test_size=0.2, random_state=42)

# --- Initialize TabNet Model ---
tabnet = TabNetRegressor(
    n_d=64, n_a=64, n_steps=5, gamma=1.3,
    lambda_sparse=1e-4, optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=1e-3), mask_type='entmax',
    seed=42, verbose=0
)

# --- Define Evaluate Function ---
def evaluate_tabnet(X_train, y_train, X_test, y_test, source="Real"):
    tabnet.fit(
        X_train=X_train, y_train=y_train.values.reshape(-1, 1),
        eval_set=[(X_test, y_test.values.reshape(-1, 1))],
        eval_name=["val"], eval_metric=["mae"],
        max_epochs=300, patience=30,
        batch_size=256, virtual_batch_size=64
    )

    # Predict and evaluate the model
    y_pred = tabnet.predict(X_test).flatten()
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mae_score_10pt = 10 * (1 - min(mae / 10, 1))
    print(f"\n TabNet Evaluation ({source}):")
    print(f"R² Score: {r2:.4f}")
    print(f"MAE: {mae:.2f}")
    print(f"MAE Score (out of 10): {mae_score_10pt:.2f}")

# --- Evaluate TabNet on Real Data Only (No Synthetic) ---
evaluate_tabnet(X_test, y_test, X_test, y_test, source="Real")


Epoch [1/200], D Loss: 0.9215498566627502, G Loss: 0.664091169834137
Epoch [2/200], D Loss: 0.7554493546485901, G Loss: 0.7615674734115601
Epoch [3/200], D Loss: 0.5539394021034241, G Loss: 0.9908871650695801
Epoch [4/200], D Loss: 0.38122060894966125, G Loss: 1.3377560377120972
Epoch [5/200], D Loss: 0.20939260721206665, G Loss: 1.9080744981765747
Epoch [6/200], D Loss: 0.17989492416381836, G Loss: 2.0276362895965576
Epoch [7/200], D Loss: 0.12312617897987366, G Loss: 2.3526039123535156
Epoch [8/200], D Loss: 0.07418923079967499, G Loss: 2.809110641479492
Epoch [9/200], D Loss: 0.0693134292960167, G Loss: 3.242720365524292
Epoch [10/200], D Loss: 0.05988695099949837, G Loss: 3.135200262069702
Epoch [11/200], D Loss: 0.027061617001891136, G Loss: 3.7924482822418213
Epoch [12/200], D Loss: 0.0749431923031807, G Loss: 3.6136670112609863
Epoch [13/200], D Loss: 0.11158554255962372, G Loss: 3.81915020942688
Epoch [14/200], D Loss: 0.023848913609981537, G Loss: 3.897347927093506
Epoch [15/2

/opt/conda/miniconda3/envs/keras-torch-310/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



 TabNet Evaluation (Real):
R² Score: 0.9505
MAE: 0.10
MAE Score (out of 10): 9.90


In [4]:
real_df.columns

Index(['Year', 'Cocoa Percent', 'Company Location', 'Rating (Out of 10)',
       'Domain Code', 'Country  Code', 'Element Code', 'Months Code', 'Months',
       'Temperature change on land Value in °c', 'Annual precipitation'],
      dtype='object')